Consider the n-dimensional linear system of ordinary differential equations:
$$x'(t)=A(t)x(t)$$
where in addition, the matrix $A(t)$ is periodic with period $T$, i.e. $A(t+T)=A(t)$ for all real values of $t$.
Let $Φ(t)$ denote the principal fundamental matrix solution such that the columns
of $Φ(t)$ are linearly independent, and $Φ(0)=I$. Then, Floquet's theorem decomposes the principal
fundamental matrix as the product of a periodic matrix $P(t)$ with period $T$ and an exponential matrix $e^{tB}$.



In [1]:
using Plots, ApproxFun

In [2]:
function Base.expm{T<:Number}(A::Matrix{T},f::Fun)
    D,V = eig(A)
    fC = D == real(D) ? fill(f,n) : fill(f+0im,n)
    for i = 1:length(D)
        fC[i] = Fun(t->exp(D[i]*fC[i](t)),domain(f))
    end
    V*diagm(fC)*inv(V)
end;

Consider the system of coupled harmonic oscillators with periodic parameterized excitation:
     $$x'' + (1+a \cos(2t))x = x-y$$
     $$y'' + (1+a \cos(2t))y = y-x$$
We calculate the Principal Fundamental Matrix $Φ$.

In [3]:
T = π;a=0.15
t = Fun(identity,0..T)
d=domain(t)
D=Derivative(d)
B=ldirichlet(d)
n=4
A=[ diagm(fill(ldirichlet(d),n));
    D             -I  0I            0I;
   (2+a*cos(2t))   D  -I            0I;
   0I             0I   D            -I;
   -I             0I  (2+a*cos(2t))  D]

# we need eye(n) for the initial condition and zeros(n,n) for the
# right-hand side of the ODE

Φ = A\eye(2n,n);  

It can be further expressed as $Φ(t) = P(t) e^{tB}$ where $P(t)$ is a periodic matrix, and $B$ is constant.
Using the values of $Φ(T)$:

In [4]:
ΦT = Φ(T)

4×4 Array{Float64,2}:
 -0.170879  -0.148885  -0.836059   0.265569
  0.732284  -0.170879  -0.612945  -0.836059
 -0.836059   0.265569  -0.170879  -0.148885
 -0.612945  -0.836059   0.732284  -0.170879

we calculate the matrix $B$ and the Floquet exponents and multipliers.

In [5]:
B = logm(ΦT)/T
D,V = eig(B)

println("Floquet Exponents = \n")
[println(D[i]) for i=1:n]
println("\nFloquet Multipliers = \n")
[println(exp(D[i]*T)) for i=1:n];

Floquet Exponents = 

-1.6653345369377348e-16 - 0.26835469053190525im
-8.630249292984615e-17 + 0.2683546905319051im
0.03747531973228935 + 0.9999999999999997im
-0.037475319732288806 + 1.0im

Floquet Multipliers = 

0.6651802570066604 - 0.7466828146465888im
0.665180257006661 + 0.7466828146465889im
-1.1249427991479373 + 1.1369156774202889e-15im
-0.8889340869219566 + 1.0886302842019492e-16im


We can build the matrix exponential $e^{-tB}$ and use it to find the periodic matrix $P(t)$.
Periodicity is numerically confirmed by `Fun` instantiation with low number of coefficients.

In [6]:
PI = Φ*expm(B,-t)
P = Fun(t->PI(t),PeriodicInterval(d))
P|>ncoefficients

223

With the matrix B and the periodic matrix P(t), we can construct 
the solution with any initial conditions for as long as we want!

In [7]:
t = Fun(identity,0..10T)
d = domain(t)
x0 = rand(n)

PI = Fun(t->P(t),d)
xsol = real(PI*expm(B,t)*x0)

plot(mat(xsol))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 -1 
 
 
 0 
 
 
 1 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 596.063,284.418 596.058,284.451 596.049,284.519 596.034,284.62 596.015,284.754 595.991,284.922 595.963,285.123 595.929,285.357 595.891,285.625 595.848,285.925 
 595.8,286.258 595.748,286.623 595.69,287.021 595.628,287.451 595.561,287.912 595.49,288.405 595.413,288.929 595.332,289.483 595.246,290.068 595.155,290.683 
 595.06,291.327 594.959,292.001 594.854,292.703 594.745,293.433 594.63,294.191 594.511,294.975 594.387,295.787 594.258,296.624 594.124,297.486 593.986,298.373 
 593.843,299.283 593.695,300.217 593.543,301.173 593.385,302.151 593.223,303.149 593.057,304.168 592.885,305.205 592.709,306.261 592.528,307.334 592.342,308.424 
 592.152,309.529 591.957,310.648 591.757,311.78 591.553,312.925 591.343,314.081 591.129,315.247 590.911,316.421 590.688,317.603 590.46,318.792 590.227,319.985 
 589.99,321.183 589.747,322.382 589.501,323.583 589.249,324.784 588.993,325.983 588.733,327.179 588.467,328.371 588.197,329.556 587.922,330.734 587.643,331.903 
 587.359,333.061 587.071,334.207 586.777,335.339 586.479,336.456 586.177,337.555 585.87,338.636 585.558,339.696 585.242,340.735 584.921,341.749 584.595,342.738 
 584.265,343.699 583.931,344.631 583.591,345.533 583.248,346.401 582.899,347.236 582.546,348.034 582.189,348.793 581.827,349.513 581.46,350.192 581.089,350.826 
 580.713,351.415 580.333,351.957 579.948,352.45 579.559,352.892 579.165,353.281 578.767,353.615 578.364,353.892 577.957,354.111 577.545,354.27 577.129,354.366 
 576.709,354.397 576.284,354.363 575.854,354.26 575.42,354.087 574.982,353.841 574.539,353.522 574.091,353.126 573.64,352.652 573.184,352.097 572.723,351.46 
 572.258,350.738 571.789,349.93 571.315,349.032 570.837,348.043 570.355,346.961 569.868,345.783 569.377,344.507 568.882,343.13 568.382,341.651 567.878,340.066 
 567.37,338.374 566.857,336.571 566.34,334.656 565.819,332.627 565.293,330.479 564.763,328.212 564.229,325.822 563.691,323.308 563.149,320.666 562.602,317.894 
 562.051,314.991 561.496,311.953 560.936,308.78 560.372,305.468 559.805,302.016 559.233,298.422 558.657,294.684 558.076,290.802 557.492,286.775 556.903,282.6 
 556.31,278.279 555.714,273.81 555.113,269.194 554.508,264.432 553.898,259.524 553.285,254.471 552.668,249.277 552.047,243.942 551.421,238.471 550.792,232.866 
 550.158,227.131 549.521,221.273 548.879,215.295 548.234,209.204 547.585,203.007 546.931,196.712 546.274,190.326 545.612,183.859 544.947,177.32 544.278,170.72 
 543.605,164.07 542.928,157.381 542.247,150.666 541.562,143.938 540.873,137.211 540.181,130.499 539.484,123.817 538.784,117.18 538.08,110.603 537.372,104.103 
 536.66,97.6951 535.945,91.3963 535.226,85.2229 534.503,79.1914 533.776,73.3182 533.045,67.6197 532.311,62.112 531.573,56.811 530.831,51.7321 530.086,46.8903 
 529.337,42.3002 528.584,37.9757 527.828,33.9301 527.068,30.1761 526.304,26.7254 525.537,23.5892 524.766,20.7777 523.992,18.3003 523.214,16.1656 522.432,14.3811 
 521.647,12.9536 520.858,11.889 520.066,11.192 519.27,10.8668 518.471,10.9164 517.668,11.3432 516.862,12.1486 516.052,13.333 515.239,14.8963 514.422,16.8373 
 513.602,19.1543 512.779,21.8445 511.952,24.9047 511.122,28.3305 510.288,32.1171 509.451,36.2586 508.611,40.7487 507.768,45.5799 506.921,50.7441 506.071,56.2323 
 505.217,62.0345 504.36,68.1399 503.5,74.5366 502.637,81.2118 501.771,88.1516 500.901,95.341 500.028,102.764 499.152,110.403 498.273,118.24 497.391,126.254 
 496.505,134.427 495.617,142.734 494.725,151.153 493.83,159.66 492.932,168.23 492.031,176.836 491.127,185.452 490.22,194.049 489.31,202.599 488.397,211.074 
 487.481,219.445 486.562,227.682 485.64,235.758 484.715,243.644 483.787,251.312 482.856,258.736 481.923,265.89 480.986,272.75 480.047,279.294 479.105,285.499 
 478.16,291.349 477.212,296.824 476.261,301.912 475.307,3